# Skyfield Almanach
Voici les importations et les objets qui piloteront les exemples ci-dessous :

In [2]:
from skyfield import almanac, eclipselib
from skyfield.api import N, S, E, W, load, wgs84, Star
from skyfield.earthlib import refraction
from skyfield.framelib import ecliptic_frame

ts = load.timescale()
eph = load('de421.bsp')
sun = eph['Sun']
cherbourg = wgs84.latlon(49.6386 * N, 1.6163 * W)
observer = eph['Earth'] + cherbourg

### Temps d’arrondi à la minute près

In [3]:
from datetime import datetime, timedelta
import pytz

def nearest_minute(dt):
    return (dt + timedelta(seconds=30)).replace(second=0, microsecond=0)

# Utiliser l'heure actuelle avec le fuseau horaire de Paris
paris_tz = pytz.timezone('Europe/Paris')

t = datetime.now(paris_tz)

dt = nearest_minute(t)
print(dt.strftime('%Y-%m-%d %H:%M'))



2024-09-05 18:44


Les résultats devraient alors concorder avec les tableaux produits par l’USNO et utilisés dans Skyfield

## Levers et couchers

### Lever et coucher du soleil

In [4]:
# Définir les temps de début et de fin pour le calcul
t0 = ts.utc(t.year, t.month, t.day)
t1 = ts.utc(t0.utc_datetime() + timedelta(days=1))

# Calculer les heures de lever et de coucher du soleil
f_sun = almanac.sunrise_sunset(eph, cherbourg)
times_sun, events_sun = almanac.find_discrete(t0, t1, f_sun)

for t, e in zip(times_sun, events_sun):
    rounded_time = nearest_minute(t.utc_datetime())
    rounded_time_paris = rounded_time.astimezone(paris_tz)
    print(f"{'Sunrise' if e else 'Sunset'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Sunrise: 2024-09-05 07:28
Sunset: 2024-09-05 20:41


#### Midi solaire

In [5]:
now = paris_tz.localize(datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + timedelta(days=1)


t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)

f = almanac.meridian_transits(eph, eph['Sun'], cherbourg)
times, events = almanac.find_discrete(t0, t1, f)

# Select transits instead of antitransits.
times = times[events == 1]

tzero = times[0]
tstr = str(tzero.astimezone(paris_tz))[:19]
print('Solar noon:', tstr)

Solar noon: 2024-09-05 14:04:59


In [6]:
# from skyfield.api import load
# from skyfield import almanac

ts = load.timescale()

# Utiliser l'heure actuelle avec le fuseau horaire de Paris
paris_tz = pytz.timezone('Europe/Paris')
t = datetime.now(paris_tz)

# Convertir l'objet datetime en une instance de Time
t_skyfield = ts.utc(t.year, t.month, t.day, t.hour, t.minute, t.second)

# Calculer la phase de la lune
phase = almanac.moon_phase(eph, t_skyfield)

# Afficher la phase de la lune en degrés
print('Moon phase: {:.1f} degrees'.format(phase.degrees))
print(t)

Moon phase: 29.3 degrees
2024-09-05 18:44:29.443307+02:00


### Lever et coucher de la Lune

In [7]:
# Calculer les heures de lever et de coucher de la lune
def moonrise_moonset(eph, location):
    t, y = almanac.find_discrete(t0, t1, almanac.risings_and_settings(eph, eph['Moon'], location))
    return t, y

times_moon, events_moon = moonrise_moonset(eph, cherbourg)

for t, e in zip(times_moon, events_moon):
    rounded_time = nearest_minute(t.utc_datetime())
    rounded_time_paris = rounded_time.astimezone(paris_tz)
    print(f"{'Moonrise' if e else 'Moonset'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Moonrise: 2024-09-05 09:52
Moonset: 2024-09-05 21:23


### Lever et coucher des planètes

In [8]:
# Calculer les heures de lever et de coucher des planètes
planets = {
    'Mercury': 199,
    'Venus': 299,
    'Mars': 499,
    'Jupiter': 5,
    'Saturn': 6,
    'Uranus': 7,
    'Neptune': 8
}

for planet, code in planets.items():
    f_planet = almanac.risings_and_settings(eph, eph[code], cherbourg)
    times_planet, events_planet = almanac.find_discrete(t0, t1, f_planet)
    for t, e in zip(times_planet, events_planet):
        rounded_time = nearest_minute(t.utc_datetime())
        rounded_time_paris = rounded_time.astimezone(paris_tz)
        print(f"{planet} {'rise' if e else 'set'}: {rounded_time_paris.strftime('%Y-%m-%d %H:%M')}")


Mercury rise: 2024-09-05 05:49
Mercury set: 2024-09-05 20:05
Venus rise: 2024-09-05 09:47
Venus set: 2024-09-05 21:30
Mars rise: 2024-09-05 01:02
Mars set: 2024-09-05 17:15
Jupiter rise: 2024-09-05 00:23
Jupiter set: 2024-09-05 16:20
Saturn set: 2024-09-05 07:48
Saturn rise: 2024-09-05 20:50
Uranus set: 2024-09-05 14:28
Uranus rise: 2024-09-05 23:03
Neptune set: 2024-09-05 09:01
Neptune rise: 2024-09-05 21:07


#### Calcul avec l'angle de refraction

In [9]:
# from skyfield.earthlib import refraction

# Calculer la réfraction
r = refraction(0.0, temperature_C=20.0, pressure_mbar=1013.0)
print('Refraction at the horizon: %.2f arcminutes\n' % (r * 60.0))

# Trouver les heures de lever et de coucher de Mars
t, y = almanac.find_risings(observer, eph['Mars'], t0, t1, horizon_degrees=-r)
for rise_time in t:
    rounded_time_paris = nearest_minute(rise_time.utc_datetime().astimezone(paris_tz))
    print('Mars rises:', rounded_time_paris.strftime('%Y-%m-%d %H:%M'))

t, y = almanac.find_settings(observer, eph['Mars'], t0, t1, horizon_degrees=-r)
for set_time in t:
    rounded_time_paris = nearest_minute(set_time.utc_datetime().astimezone(paris_tz))
    print('Mars sets: ', rounded_time_paris.strftime('%Y-%m-%d %H:%M'))

Refraction at the horizon: 33.38 arcminutes

Mars rises: 2024-09-05 01:02
Mars sets:  2024-09-05 17:15


## Création d'une ascension droite et une déclinaison d'un point fixe

In [10]:
# from skyfield.api import Star

galactic_center = Star(ra_hours=(17, 45, 40.04),
                       dec_degrees=(-29, 0, 28.1))

t, y = almanac.find_risings(observer, galactic_center, t0, t1)
for rise_time in t:
    rounded_time_paris = nearest_minute(rise_time.utc_datetime().astimezone(paris_tz))
    print('The Galactic Center rises at', rounded_time_paris.strftime('%Y-%m-%d %H:%M'))

The Galactic Center rises at 2024-09-05 17:31


## Saisons

In [11]:
# t0 = ts.utc(t.year, t.month, t.day)
t2 = ts.utc(t0.utc_datetime() + timedelta(days=365))
t, y = almanac.find_discrete(t0, t2, almanac.seasons(eph))

for yi, ti in zip(y, t):
    rounded_time_paris = nearest_minute(ti.utc_datetime().astimezone(paris_tz))
    print(yi, almanac.SEASON_EVENTS[yi], rounded_time_paris.strftime('%Y-%m-%d %H:%M'))

2 Autumnal Equinox 2024-09-22 14:44
3 Winter Solstice 2024-12-21 10:21
0 Vernal Equinox 2025-03-20 10:01
1 Summer Solstice 2025-06-21 04:42


## Phases de la Lune

In [12]:
# from skyfield.api import load
# from skyfield import almanac



# Utiliser l'heure actuelle avec le fuseau horaire de Paris

t = datetime.now(paris_tz)

# Calculer la phase de la lune
phase = almanac.moon_phase(eph, t_skyfield)

# Afficher la phase de la lune en degrés
print('Moon phase: {:.1f} degrees'.format(phase.degrees))
print(t)


Moon phase: 29.3 degrees
2024-09-05 18:44:29.872296+02:00


In [13]:
# t0 = ts.utc(2018, 9, 1)
# t1 = ts.utc(2018, 9, 10)
t, y = almanac.find_discrete(t0, t1, almanac.moon_phases(eph))

print(t.utc_iso())
print(y)
print([almanac.MOON_PHASES[yi] for yi in y])

[]
[]
[]


#### Quelle est la phase de la Lune ce soir ?

In [14]:
eph = load('de421.bsp')
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']

e = earth.at(t_skyfield)
s = e.observe(sun).apparent()
m = e.observe(moon).apparent()

_, slon, _ = s.frame_latlon(ecliptic_frame)
_, mlon, _ = m.frame_latlon(ecliptic_frame)
phase = (mlon.degrees - slon.degrees) % 360.0

percent = 100.0 * m.fraction_illuminated(sun)

print('Phase (0°–360°): {0:.1f}'.format(phase))
print('Percent illuminated: {0:.1f}%'.format(percent))

Phase (0°–360°): 29.3
Percent illuminated: 6.4%


#### Quel est le diamètre angulaire de la Lune, compte tenu de son rayon ?

In [15]:
import numpy as np
from skyfield.api import Angle, load

ts = load.timescale()
time = t_skyfield

eph = load('de421.bsp')
earth, moon = eph['earth'], eph['moon']
radius_km = 3474.2

astrometric = earth.at(time).observe(moon)
ra, dec, distance = astrometric.apparent().radec()
apparent_diameter = Angle(radians=np.arcsin(radius_km / distance.km) * 2.0)
print('{:.6f} arcseconds'.format(apparent_diameter.arcseconds()))
print(apparent_diameter)
print(distance.km)
print(t_skyfield.utc_strftime('%Y-%m-%d %H:%M'))

3528.207825 arcseconds
00deg 58' 48.2"
406219.7929932135
2024-09-05 18:44


##### Quel est le diamètre angulaire d’une planète, compte tenu de son rayon ? Exemple

In [16]:
import numpy as np
from skyfield.api import Angle, load

ts = load.timescale()
time = t_skyfield

eph = load('de421.bsp')
earth, neptune = eph['earth'], eph['neptune barycenter']
radius_km = 24764.0

astrometric = earth.at(time).observe(neptune)
ra, dec, distance = astrometric.apparent().radec()
apparent_diameter = Angle(radians=np.arcsin(radius_km / distance.km) * 2.0)
print('{:.6f} arcseconds'.format(apparent_diameter.arcseconds()))
print(apparent_diameter)
print(distance.km)
print(t_skyfield.utc_strftime('%Y-%m-%d %H:%M'))

2.360998 arcseconds
00deg 00' 02.4"
4326934029.055356
2024-09-05 18:44


### Nœud lunaire

In [17]:
# t0 = ts.utc(2020, 4, 22)
# t1 = ts.utc(2020, 5, 22)
t, y = almanac.find_discrete(t0, t1, almanac.moon_nodes(eph))

print(t.utc_iso())
print(y)
print([almanac.MOON_NODES[yi] for yi in y])

['2024-09-05T05:43:23Z']
[0]
['descending']


### Opposition et conjonction

In [18]:


# Charger les échelles de temps

t0 = ts.utc(datetime.now().year, datetime.now().month, datetime.now().day)
t1 = ts.utc((datetime.now() + timedelta(days=365)).year, (datetime.now() + timedelta(days=365)).month, (datetime.now() + timedelta(days=365)).day)



# Fonction pour trouver les oppositions et conjonctions de Mars
f = almanac.oppositions_conjunctions(eph, eph['mars'])
t, y = almanac.find_discrete(t0, t1, f)

# Filtrer pour obtenir la prochaine opposition
now = ts.now()
next_opposition = None
next_conjunction = None

# for ti, yi in zip(t, y):
#     if ti > now and yi == 1:
#         next_opposition = ti
#         break

for ti, yi in zip(t, y):
    if ti > now and yi == 1:
        next_opposition = ti
        break
    if ti > now and yi == 0:
        next_conjunction = ti
        break
    

# # Afficher le résultat
# if next_opposition is not None:
#     print(f"Prochaine opposition de Mars : {next_opposition.utc_iso()}")
# else:
#     print("Aucune opposition trouvée dans la période spécifiée.")

if next_opposition is not None:
    print(f"Prochaine opposition de Mars : {next_opposition.utc_iso()}")
else:
    print("Aucune opposition trouvée dans la période spécifiée.")

if next_conjunction is not None:
    print(f"Prochaine conjonction de Mars : {next_conjunction.utc_iso()}")
else:
    print("Aucune conjonction trouvée dans la période spécifiée.")

Prochaine opposition de Mars : 2025-01-16T02:38:35Z
Aucune conjonction trouvée dans la période spécifiée.


### Transits méridiens

In [19]:
# t0 = ts.utc(2020, 11, 6)
# t1 = ts.utc(2020, 11, 8)
t = almanac.find_transits(observer, eph['Mars'], t0, t1)

print(t.utc_strftime('%Y-%m-%d %H:%M'))

['2024-09-05 07:08', '2024-09-06 07:07', '2024-09-07 07:06', '2024-09-08 07:04', '2024-09-09 07:03', '2024-09-10 07:01', '2024-09-11 07:00', '2024-09-12 06:59', '2024-09-13 06:57', '2024-09-14 06:56', '2024-09-15 06:55', '2024-09-16 06:53', '2024-09-17 06:52', '2024-09-18 06:50', '2024-09-19 06:49', '2024-09-20 06:47', '2024-09-21 06:46', '2024-09-22 06:44', '2024-09-23 06:43', '2024-09-24 06:41', '2024-09-25 06:40', '2024-09-26 06:38', '2024-09-27 06:36', '2024-09-28 06:35', '2024-09-29 06:33', '2024-09-30 06:32', '2024-10-01 06:30', '2024-10-02 06:28', '2024-10-03 06:26', '2024-10-04 06:25', '2024-10-05 06:23', '2024-10-06 06:21', '2024-10-07 06:20', '2024-10-08 06:18', '2024-10-09 06:16', '2024-10-10 06:14', '2024-10-11 06:12', '2024-10-12 06:10', '2024-10-13 06:09', '2024-10-14 06:07', '2024-10-15 06:05', '2024-10-16 06:03', '2024-10-17 06:01', '2024-10-18 05:59', '2024-10-19 05:57', '2024-10-20 05:55', '2024-10-21 05:53', '2024-10-22 05:50', '2024-10-23 05:48', '2024-10-24 05:46',

### Crépuscules

In [20]:
# Figure out local midnight
now = paris_tz.localize(datetime.now())
midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
next_midnight = midnight + timedelta(days=1)

t0 = ts.from_datetime(midnight)
t1 = ts.from_datetime(next_midnight)

f = almanac.dark_twilight_day(eph, cherbourg)
times, events = almanac.find_discrete(t0, t1, f)

previous_e = f(t0).item()
for t, e in zip(times, events):
    tstr = str(t.astimezone(paris_tz))[:16]
    if previous_e < e:
        print(tstr, ' ', almanac.TWILIGHTS[e], 'starts')
    else:
        print(tstr, ' ', almanac.TWILIGHTS[previous_e], 'ends')
    previous_e = e

2024-09-05 05:32   Astronomical twilight starts
2024-09-05 06:15   Nautical twilight starts
2024-09-05 06:55   Civil twilight starts
2024-09-05 07:28   Day starts
2024-09-05 20:40   Day ends
2024-09-05 21:13   Civil twilight ends
2024-09-05 21:53   Nautical twilight ends
2024-09-05 22:35   Astronomical twilight ends


### Éclipses lunaires

In [21]:
# from skyfield import eclipselib

# t0 = ts.utc(2019, 1, 1)
# t1 = ts.utc(2020, 1, 1)
t, y, details = eclipselib.lunar_eclipses(t0, t1, eph)

for ti, yi in zip(t, y):
    print(ti.utc_strftime('%Y-%m-%d %H:%M'),
          'y={}'.format(yi),
          eclipselib.LUNAR_ECLIPSES[yi])
    
for name, values in sorted(details.items()):
    print(f'{name:24}  {values}')

closest_approach_radians  []
moon_radius_radians       []
penumbra_radius_radians   []
penumbral_magnitude       []
umbra_radius_radians      []
umbral_magnitude          []


In [22]:
from skyfield.api import load
from skyfield.framelib import ecliptic_frame
from skyfield.searchlib import find_maxima

ts = load.timescale()
# t0 = ts.utc(2019)
# t1 = ts.utc(2022)

eph = load('de421.bsp')
sun, earth, venus = eph['sun'], eph['earth'], eph['venus']

def elongation_at(t):
    e = earth.at(t)
    s = e.observe(sun).apparent()
    v = e.observe(venus).apparent()
    return s.separation_from(v).degrees

elongation_at.step_days = 15.0

times, elongations = find_maxima(t0, t2, elongation_at)

for t, elongation_degrees in zip(times, elongations):
    e = earth.at(t)
    _, slon, _ = e.observe(sun).apparent().frame_latlon(ecliptic_frame)
    _, vlon, _ = e.observe(venus).apparent().frame_latlon(ecliptic_frame)
    is_east = (vlon.degrees - slon.degrees) % 360.0 < 180.0
    direction = 'east' if is_east else 'west'
    print('{}  {:4.1f}° {} elongation'.format(
        t.utc_strftime(), elongation_degrees, direction))

2025-01-10 05:01:36 UTC  47.2° east elongation
2025-06-01 03:28:33 UTC  45.9° west elongation
